In [7]:
import os
import json
import pandas as pd

from datetime import datetime
from utils.config import DRLConfig
from utils.drl_train import training_pipeline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
DATA_DIR = "../data/snp_new"
RETURNS_PATH = os.path.join(DATA_DIR, "returns_1d.parquet")
PRICES_PATH = os.path.join(DATA_DIR, "prices_1d.parquet")
VOLA_PATH = os.path.join(DATA_DIR, "vola_1d.parquet")

# DATA_DIR = "../data/intl"
# RETURNS_PATH = os.path.join(DATA_DIR, "returns_1d.parquet")
# PRICES_PATH = os.path.join(DATA_DIR, "prices_1d.parquet")
# VOLA_PATH = os.path.join(DATA_DIR, "vola_1d.parquet")

df_ret = pd.read_parquet(RETURNS_PATH)
df_prices = pd.read_parquet(PRICES_PATH)
df_vol = pd.read_parquet(VOLA_PATH)

In [9]:
# To view the logs:
# 1. Open a terminal or command prompt.
# 2. Navigate to the directory *containing* the `logs` directory (i.e., the root of this repository).
# 3. Run the command: `tensorboard --logdir logs/`
# 4. Open the URL provided by TensorBoard (usually http://localhost:6006/) in your web browser.

# Create timestamp for this run
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Create configuration
config = DRLConfig(
    # Window configuration
    n_windows=1,  # 10 in paper
    agents_per_window=5,  # 5 in paper
    base_start_year=2015,  # 2006 in paper
    use_previous_best_seed=False,
    # Environment parameters
    env_window_size=60,
    transaction_cost=0.0,
    initial_balance=100_000,
    reward_scaling=1.0,
    eta_dsr=1 / 252,
    # Training parameters
    n_envs=10,
    total_timesteps_per_round=7_500_000,  # 7_500_000 in paper
    n_steps_per_env=252 * 3,
    batch_size=1260,
    n_epochs=16,
    gamma=0.9,
    gae_lambda=0.9,
    clip_range=0.25,
    log_std_init=-1.0,
    # Learning rate parameters
    initial_lr=3e-4,
    final_lr=1e-5,
    # Paths
    model_save_dir=f"../models/{timestamp}",
    tensorboard_log_dir=f"../logs/{timestamp}",
    data_dir=DATA_DIR,
)

In [10]:
# Save config as JSON
from dataclasses import asdict

config_dict = asdict(config)
config_json_path = os.path.join(config.model_save_dir, f"config_{timestamp}.json")
os.makedirs(config.model_save_dir, exist_ok=True)
with open(config_json_path, "w") as f:
    json.dump(config_dict, f, indent=4)
print(f"\nConfiguration saved to: {config_json_path}")


Configuration saved to: ../models/20250701_082652/config_20250701_082652.json


In [ ]:
# Run training pipeline
results, backtest_portfolio = training_pipeline(
    drl_config=config, df_prices=df_prices, df_ret=df_ret, df_vol=df_vol
)

--- Starting Window 1/1 (Train Year Start: 2015) ---
  Train Period: 2015-01-01 to 2019-12-31
  Val Period  : 2020-01-01 to 2020-12-31
  Test Period : 2021-01-01 to 2021-12-31
  Starting with fresh random initialization
  Training Agent 1/5 with seed 0...


Output()

    Starting training for 7500000 timesteps...



Training complete. Trained for 7500000 timesteps.
TensorBoard logs for experiment 'PPO_Seed0' saved in directory: ../logs/20250701_082652
    Evaluating agent on validation set...
    Validation Mean Reward: -8.9831
    Agent saved to: ../models/20250701_082652/agent_seed0_valrew-8.98.zip
  Training Agent 2/5 with seed 1...


Output()

    Starting training for 7500000 timesteps...



Training complete. Trained for 7500000 timesteps.
TensorBoard logs for experiment 'PPO_Seed1' saved in directory: ../logs/20250701_082652
    Evaluating agent on validation set...
    Validation Mean Reward: -9.7458
    Agent saved to: ../models/20250701_082652/agent_seed1_valrew-9.75.zip
  Training Agent 3/5 with seed 2...


Output()

    Starting training for 7500000 timesteps...


In [6]:
for idx, p in backtest_portfolio.items():
    fname = f"{idx}_portfolio.csv"
    p.get_history().to_csv(os.path.join(config.model_save_dir, fname))

NameError: name 'backtest_portfolio' is not defined

In [ ]:
results_filename = f"backtest_results_summary_{timestamp}.csv"
results_save_path = os.path.join(config.model_save_dir, results_filename)

results_df = pd.DataFrame(results)
results_df.to_csv(results_save_path, index=False)
print(f"\nBacktest results summary saved to: {results_save_path}")
print("\nFinal Results DataFrame:")
results_df.head()


Backtest results summary saved to: ../models/20250701_004204/backtest_results_summary_20250701_004204.csv

Final Results DataFrame:


,window,best_agent_path,n_eval_episodes,mean_reward,std_reward,Annual return,Cumulative returns,Annual volatility,Sharpe ratio,Calmar ratio,Stability,Max drawdown,Omega ratio,Sortino ratio,Skew,Kurtosis,Tail ratio,Daily value at risk (95%),Portfolio turnover (in %),final_portfolio_value_first_episode
0,1,agent_seed0_valrew15,1,36.708522,0.0,0.059012,0.058530,0.180260,0.409353,0.528551,0.847271,-0.111649,1.076760,0.484042,-1.560477,10.749818,0.929536,-0.017837,0.695361,105853
1,2,agent_seed3_valrew36,1,18.755455,0.0,0.239067,0.235909,0.079216,2.746644,10.983807,0.926598,-0.021765,1.593735,4.821020,0.625763,4.000017,1.324281,-0.006126,0.508986,123591
